<a id='clustering'></a>
## 5. Clustering

In [ ]:
# Find optimal number of clusters for KMeans
print("Finding optimal number of clusters for KMeans...")
k_range = range(2, 11)
optimal_k_elbow, optimal_k_silhouette, inertia_values, silhouette_values = find_optimal_k(
    X_reduced, 
    k_range=k_range, 
    random_state=config['clustering']['kmeans']['random_state']
)

print(f"Optimal k based on elbow method: {optimal_k_elbow}")
print(f"Optimal k based on silhouette score: {optimal_k_silhouette}")

# Plot elbow method results
fig = plot_elbow_method(k_range, inertia_values, silhouette_values)
plt.show()

In [ ]:
# Apply clustering using the optimal number of clusters
print(f"Applying {config['clustering']['method']} clustering...")

# Update config with optimal number of clusters if using KMeans
if config['clustering']['method'] == 'kmeans':
    # Choose the optimal k based on silhouette score
    config['clustering']['kmeans']['n_clusters'] = optimal_k_silhouette
    print(f"Using optimal number of clusters: {optimal_k_silhouette}")

# Get method-specific parameters
method = config['clustering']['method']
method_params = config['clustering'][method]

# Apply clustering
labels, model, metrics = cluster_data(
    X_reduced,
    method=method,
    **method_params
)

# Print metrics
print("\nClustering metrics:")
for metric_name, metric_value in metrics.items():
    print(f"- {metric_name}: {metric_value:.4f}")

In [ ]:
# Visualize clustering results
print("Visualizing clustering results...")

# Get centroids if using KMeans
centroids = None
if method == 'kmeans':
    centroids = model.cluster_centers_

# Plot clusters
fig = plot_clusters_2d(
    X_reduced, 
    labels, 
    centroids=centroids, 
    title=f'{method.upper()} Clustering Results',
    figsize=config['visualization']['figsize'],
    alpha=config['visualization']['alpha'],
    s=config['visualization']['s']
)
plt.show()

In [ ]:
# Compare different clustering methods
print("Comparing different clustering methods...")

# Define methods to compare
clustering_methods = ['kmeans', 'dbscan', 'agglomerative']

# Define parameters for each method
clustering_params = {
    'kmeans': {
        'n_clusters': optimal_k_silhouette,
        'random_state': config['clustering']['kmeans']['random_state']
    },
    'dbscan': {
        'eps': config['clustering']['dbscan']['eps'],
        'min_samples': config['clustering']['dbscan']['min_samples']
    },
    'agglomerative': {
        'n_clusters': optimal_k_silhouette,
        'linkage': config['clustering']['agglomerative']['linkage']
    }
}

# Apply each method
clustering_results = compare_clustering_methods(
    X_reduced,
    methods=clustering_methods,
    **clustering_params
)

# Extract labels for visualization
labels_dict = {}
for method_name, (labels_method, _, _) in clustering_results.items():
    labels_dict[method_name.upper()] = labels_method

In [ ]:
# Visualize clustering results for each method
from src.visualization import compare_dimensionality_reduction_methods as compare_vis

# Create a dictionary with the same reduced data for each method
X_dict_for_vis = {method_name.upper(): X_reduced for method_name in clustering_methods}

# Plot clustering results
fig = compare_vis(X_dict_for_vis, labels_dict)
plt.show()

<a id='evaluation'></a>
## 6. Evaluation and Visualization

In [ ]:
# Compare clustering results using evaluation metrics
print("Comparing clustering results using evaluation metrics...")
comparison_df = compare_clustering_results(X_reduced, clustering_results)
comparison_df

In [ ]:
# Select the best clustering method based on silhouette score
best_method = comparison_df['silhouette'].idxmax()
print(f"Best clustering method based on silhouette score: {best_method}")

# Get the labels from the best method
best_labels = clustering_results[best_method.lower()][0]

# Create an interactive visualization of the best clustering result
if customer_ids is not None:
    hover_data = pd.DataFrame({'customer_id': customer_ids})
else:
    hover_data = None

fig = create_interactive_scatter(
    X_reduced, 
    best_labels, 
    hover_data=hover_data, 
    title=f'{best_method} Clustering Results'
)
fig.show()

<a id='analysis'></a>
## 7. Cluster Analysis and Interpretation

In [ ]:
# Analyze clusters
print("Analyzing clusters...")

# Get original feature names
feature_names = df.columns.tolist()

# Analyze clusters using the best labels
cluster_profiles = analyze_clusters(df.values, best_labels, feature_names)
cluster_profiles

In [ ]:
# Visualize cluster profiles
fig = plot_cluster_profiles(cluster_profiles)
plt.show()

In [ ]:
# Generate human-readable labels for clusters
cluster_labels = generate_cluster_labels(cluster_profiles)

print("Cluster labels:")
for cluster_id, label in cluster_labels.items():
    size = cluster_profiles.loc[cluster_id, 'Size'] if 'Size' in cluster_profiles.columns else 'N/A'
    print(f"Cluster {cluster_id} ({size} customers): {label}")

In [ ]:
# Create a DataFrame with customer IDs and cluster labels
if customer_ids is not None:
    customer_clusters = pd.DataFrame({
        'customer_id': customer_ids,
        'cluster': best_labels,
        'cluster_name': [cluster_labels[label] for label in best_labels]
    })
    customer_clusters.head(10)

<a id='edge_cases'></a>
## 8. Edge Cases and Robustness

In [ ]:
# Test robustness with small dataset
print("Testing robustness with small dataset...")

# Generate a small dataset
    # Load a small subset of the real data
    df_full, _ = load_online_retail_data("None")
    small_df = df_full.sample(n=30, random_state=42)
print(f"Small dataset shape: {small_df.shape}")

# Preprocess the small dataset
small_df_processed = preprocess_data(small_df.drop('customer_id', axis=1), config=config['preprocessing'])

# Apply dimensionality reduction
small_X_reduced, _, _ = reduce_dimensions(
    small_df_processed.values,
    method=config['dimensionality_reduction']['method'],
    n_components=config['dimensionality_reduction']['n_components'],
    random_state=config['dimensionality_reduction']['random_state'],
    **config['dimensionality_reduction'][config['dimensionality_reduction']['method']]
)

# Apply clustering
small_labels, small_model, small_metrics = cluster_data(
    small_X_reduced,
    method=config['clustering']['method'],
    **config['clustering'][config['clustering']['method']]
)

# Print metrics
print("\nClustering metrics for small dataset:")
for metric_name, metric_value in small_metrics.items():
    print(f"- {metric_name}: {metric_value:.4f}")

# Visualize clustering results
fig = plot_clusters_2d(
    small_X_reduced, 
    small_labels, 
    title=f'{config["clustering"]["method"].upper()} Clustering Results (Small Dataset)',
    figsize=config['visualization']['figsize'],
    alpha=config['visualization']['alpha'],
    s=config['visualization']['s']
)
plt.show()

In [ ]:
# Test robustness with high-dimensional noisy data
print("Testing robustness with high-dimensional noisy data...")

# Generate a dataset with additional noisy features
    # Load data and add noise
    df_full, _ = load_online_retail_data("None")
    noisy_df = df_full.sample(n=200, random_state=42).copy()
    # Add noise to numerical columns
    for col in noisy_df.select_dtypes(include=["float64", "int64"]).columns:
        noise = np.random.normal(0, noisy_df[col].std() * 0.2, size=len(noisy_df))
        noisy_df[col] = noisy_df[col] + noise

# Add noisy features
for i in range(10):
    noisy_df[f'noise_{i}'] = np.random.normal(0, 1, size=len(noisy_df))

print(f"Noisy dataset shape: {noisy_df.shape}")

# Preprocess the noisy dataset
noisy_df_processed = preprocess_data(noisy_df.drop('customer_id', axis=1), config=config['preprocessing'])

# Apply dimensionality reduction
noisy_X_reduced, _, _ = reduce_dimensions(
    noisy_df_processed.values,
    method=config['dimensionality_reduction']['method'],
    n_components=config['dimensionality_reduction']['n_components'],
    random_state=config['dimensionality_reduction']['random_state'],
    **config['dimensionality_reduction'][config['dimensionality_reduction']['method']]
)

# Apply clustering
noisy_labels, noisy_model, noisy_metrics = cluster_data(
    noisy_X_reduced,
    method=config['clustering']['method'],
    **config['clustering'][config['clustering']['method']]
)

# Print metrics
print("\nClustering metrics for noisy dataset:")
for metric_name, metric_value in noisy_metrics.items():
    print(f"- {metric_name}: {metric_value:.4f}")

# Visualize clustering results
fig = plot_clusters_2d(
    noisy_X_reduced, 
    noisy_labels, 
    title=f'{config["clustering"]["method"].upper()} Clustering Results (Noisy Dataset)',
    figsize=config['visualization']['figsize'],
    alpha=config['visualization']['alpha'],
    s=config['visualization']['s']
)
plt.show()

In [ ]:
# Test robustness with imbalanced clusters
print("Testing robustness with imbalanced clusters...")

# Generate a dataset with imbalanced clusters
np.random.seed(42)
n_samples = 500

# Generate cluster 1 (80% of data)
cluster1_size = int(0.8 * n_samples)
cluster1_data = np.random.normal(0, 1, size=(cluster1_size, 2))

# Generate cluster 2 (15% of data)
cluster2_size = int(0.15 * n_samples)
cluster2_data = np.random.normal(5, 1, size=(cluster2_size, 2))

# Generate cluster 3 (5% of data)
cluster3_size = n_samples - cluster1_size - cluster2_size
cluster3_data = np.random.normal(-5, 1, size=(cluster3_size, 2))

# Combine clusters
imbalanced_data = np.vstack([cluster1_data, cluster2_data, cluster3_data])
print(f"Imbalanced dataset shape: {imbalanced_data.shape}")

# Apply clustering
imbalanced_labels, imbalanced_model, imbalanced_metrics = cluster_data(
    imbalanced_data,
    method=config['clustering']['method'],
    **config['clustering'][config['clustering']['method']]
)

# Print metrics
print("\nClustering metrics for imbalanced dataset:")
for metric_name, metric_value in imbalanced_metrics.items():
    print(f"- {metric_name}: {metric_value:.4f}")

# Visualize clustering results
fig = plot_clusters_2d(
    imbalanced_data, 
    imbalanced_labels, 
    title=f'{config["clustering"]["method"].upper()} Clustering Results (Imbalanced Dataset)',
    figsize=config['visualization']['figsize'],
    alpha=config['visualization']['alpha'],
    s=config['visualization']['s']
)
plt.show()

<a id='conclusion'></a>
## 9. Conclusion

### Summary of Findings

In this notebook, we implemented an unsupervised learning pipeline for customer segmentation in an e-commerce scenario. The pipeline includes:

1. **Data Preprocessing**: We handled missing values, removed outliers, scaled features, and encoded categorical variables.

2. **Dimensionality Reduction**: We compared PCA, Kernel PCA, MDS, and UMAP for reducing the dimensionality of the data.

3. **Clustering**: We applied KMeans, DBSCAN, and Agglomerative Clustering to identify distinct customer groups.

4. **Evaluation**: We evaluated the clustering results using silhouette score, Davies-Bouldin index, and visual inspection.

5. **Interpretation**: We analyzed the cluster profiles and generated human-readable labels for each cluster.

6. **Robustness**: We tested the pipeline on small datasets, high-dimensional noisy data, and imbalanced clusters.

### Key Insights

- The optimal number of clusters for this dataset was determined to be [optimal_k_silhouette].
- The best performing clustering method was [best_method] based on the silhouette score.
- We identified distinct customer segments with the following characteristics: [list cluster labels].
- The pipeline demonstrated robustness to various edge cases, including small datasets, noisy data, and imbalanced clusters.

### Business Applications

These customer segments can be used for:

1. **Targeted Marketing**: Tailoring marketing campaigns to specific customer segments.
2. **Personalization**: Customizing the user experience based on the segment a customer belongs to.
3. **Customer Retention**: Developing strategies to retain customers in high-value segments.
4. **Product Recommendations**: Recommending products based on the preferences of similar customers in the same segment.

### Future Work

1. **Feature Engineering**: Develop more sophisticated features to better capture customer behavior.
2. **Time-Series Analysis**: Incorporate temporal patterns in customer behavior.
3. **Semi-Supervised Learning**: Use labeled data to guide the clustering process.
4. **Interactive Dashboard**: Build a Streamlit or Dash app for interactive exploration of customer segments.